In [1]:
import os
os.chdir('../../..')

In [3]:
import convokit
import pickle
import json

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
os.chdir('convokit/tensors')

In [6]:
with open('data/rank_to_factors.p', 'rb') as f:
    rank_to_factors = pickle.load(f)

In [7]:
with open('data/hg_features.p', 'rb') as f:
    hg_features = pickle.load(f)

with open('data/subreddits.p', 'rb') as f:
    subreddits = pickle.load(f)

In [11]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()

In [12]:
def get_anomalous_points(factor_full, idx):
    factor = factor_full[:, idx]
    reshaped = factor.reshape((factor.shape[0], 1))
    scaled = scaler.fit_transform(reshaped)
    pos_pts = np.argwhere(scaled.reshape(factor.shape[0]) > 1.5).flatten()
    neg_pts = np.argwhere(scaled.reshape(factor.shape[0]) < -1.5).flatten()
    return pos_pts, neg_pts

In [13]:
time_factor = rank_to_factors[9][0] # (9, 9)
thread_factor = rank_to_factors[9][1] # (10000, 9)
feature_factor = rank_to_factors[9][2] # (164, 9)

In [ ]:
with open('data/thread_ids.p', 'rb') as f:
    thread_ids = pickle.load(f)

In [22]:
from convokit import Corpus, download
corpus = Corpus(filename=download('reddit-corpus-small'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/reddit-corpus-small


In [23]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [24]:
corpus = corpus.reindex_conversations(top_level_comment_ids)

In [25]:
threads = [corpus.get_conversation(thread_id) for thread_id in thread_ids]

## Inspecting threads from Factor 1

In [ ]:
pos_threads, neg_threads = get_anomalous_points(thread_factor, 0)

In [33]:
def get_convo_details(convo):
    print("Subreddit: {}".format(convo.meta['original_convo_meta']['subreddit']))
    convo.print_conversation_structure()

In [35]:
import random

In [38]:
random.sample(list(pos_threads), 10)

[5344, 1006, 6609, 5317, 5128, 4897, 2366, 8101, 258, 9881]

In [39]:
get_convo_details(threads[6609])

Subreddit: buildapc
Stew_a_jew
    podboi
        Stew_a_jew
            podboi
                Stew_a_jew
                    podboi
                        Stew_a_jew
                            podboi
                                Stew_a_jew
                                    podboi
                                        Stew_a_jew
                                        Stew_a_jew
                                            podboi
            Stew_a_jew


In [40]:
get_convo_details(threads[2366])

Subreddit: Christianity
northstardim
    makutamakaveli
        northstardim
            makutamakaveli
                northstardim
                    makutamakaveli
                        northstardim
                            makutamakaveli
                                northstardim
                                    makutamakaveli
                                yemayknow
                                    makutamakaveli
                                        yemayknow
                                            makutamakaveli
                                                makutamakaveli
                                                yemayknow
                                                    makutamakaveli
                                                        yemayknow
                                                            makutamakaveli
                                                                yemayknow
                                                   

In [41]:
get_convo_details(threads[8101])

Subreddit: buildapc
kvswim
    Catman360
        kvswim
            Catman360
                kvswim
                    Catman360
                        kvswim
                            Catman360
                                kvswim
                                    Catman360
                                        kvswim
                                            Catman360


Looks like dyadic reciprocity.

## Inspecting threads from Factor 9

In [55]:
pos_threads9, neg_threads9 = get_anomalous_points(thread_factor, 8)

In [56]:
random.sample(list(pos_threads9), 10)

[5739, 334, 9462, 6566, 7229, 9118, 6916, 6318, 1335, 4666]

In [66]:
threads[9118].print_conversation_structure(lambda x: x.text)

Can you link the inspiration cover? I can’t find it
    * [This one.](https://i.imgur.com/zFUmfpc.jpg) 

* [Into the Spider-verse poster does something similar too.](https://i.imgur.com/DVQHBj7.jpg)
        Awesome! Yours included
        God. That’s a powerful image.
        [there's this one from into the spider verse aswell](https://www.mpaa.org/wp-content/uploads/2017/12/brazil-comic-con-cbf_mkt_firstlook_comp_004_jkt_dg_rgb.jpg)
        [Thought it was more inspired by this from Into the Spider-Verse](https://www.cartoonbrew.com/wp-content/uploads/2017/12/spiderman_spiderverse.jpg)
        Thank you.
        Damn dude you took a screen shot of a video game that trumps the cover it was imitating. Incredible. 
        Your title could have used the weird book, I'm sure many of us thought the film. 
        Into the Spider-Verse is gonna be fantastic. 
        All I thought of when I first saw your pic was that photo from 9/11 called The Falling Man.... It's a bit depressing so I won

In [57]:
get_convo_details(threads[9118])

Subreddit: gaming
tallest_chris
    ulsafar
        tallest_chris
        jburtson
        CUI2I2EY
        ram-ok
        PKMNTrainerMark
        mushroomking311
        CageAndBale
        KilowogTrout
        My_Name_Is_Steven


In [58]:
get_convo_details(threads[7229])

Subreddit: LifeProTips
NewShadowGuy
    Wootimonreddit
        NewShadowGuy
            Wootimonreddit
                NewShadowGuy
                bigbadbosp
        westbee
            Wootimonreddit
                Destination_Fucked
            hairyblackhole
        SpeckleCoatedYak


In [62]:
get_convo_details(threads[9462])

Subreddit: MensRights
skepticalbob
    Imnotmrabut
        skepticalbob
            Imnotmrabut
                skepticalbob
                    Imnotmrabut
                        skepticalbob
                            Imnotmrabut
                                skepticalbob
                                    Imnotmrabut
                            grandmasbroach


In [59]:
random.sample(list(neg_threads9), 10)

[2741, 3059, 2547, 1267, 1017, 3173, 7409, 2098, 7004, 996]

In [60]:
get_convo_details(threads[1267])

Subreddit: aww
AmoreBestia
    HeyPScott
    HarryCochrane
        isolophobichermit
            burnsito
    hehyougotme
        ApplesPeaches
        new-it-grad
    LiberatedCapsicum
        Dovah-Bot
    Nosefairy
        AmoreBestia
            aleksem
                AmoreBestia
                    aleksem
        PoIwho
        DoubleTapJ
            Nosefairy
                DoubleTapJ
    inkflamingo
    anefisenuf


In [61]:
get_convo_details(threads[1017])

Subreddit: Android
cjandstuff
    DrDan21
    Nestramutat-
    cxu1993
    [deleted]
    lifeiswater
        Jacko3000
            electricblues42
                Jacko3000
    CharlesHipster
    tossin


## Factor 7

In [67]:
pos_threads7, neg_threads7 = get_anomalous_points(thread_factor, 6)

In [68]:
random.sample(list(pos_threads7), 10)

[1981, 6876, 29, 4152, 6298, 7597, 4568, 8457, 4844, 3043]

In [69]:
get_convo_details(threads[1981])

Subreddit: Games
Nzash
    AnimaLepton
    [deleted]
    SquilliamFancyson412
        DexRogue
            Sir__Walken
                teleporterdown
                TSPhoenix
                    SquilliamFancyson412
                    Cr0nq
                        TSPhoenix
            Lugonn
                SquilliamFancyson412
                    brain_fuckler
                        SquilliamFancyson412
    TSPhoenix


In [76]:
for idx, utt in enumerate(threads[7597].get_chronological_utterance_list()):
    utt.meta['order'] = idx

In [79]:
thread_ids[7597]

'e5shplt'

In [78]:
threads[7597].print_conversation_structure(lambda utt: str(utt.meta['order']) + ". " + utt.user.id)

0. twentyitalians
    1. thatgirlwithbigboobs
        8. pylestothemax
            18. trebory6
                23. pylestothemax
            35. ismailismail
    2. RockstarSuicide
        12. 1stOnRt1
            14. oneELECTRIC
            27. screw_arc
                30. 1stOnRt1
                    33. JasonSteakums
            28. RockstarSuicide
                31. 1stOnRt1
                    32. RockstarSuicide
    3. Fanatical_Idiot
        6. TokeyWeedtooth
            7. Fanatical_Idiot
            9. da_bbq
                11. Martel732
                    15. [deleted]
                16. TokeyWeedtooth
            10. MC_Fillius_Dickinson
                17. TokeyWeedtooth
                    20. MC_Fillius_Dickinson
                        21. TokeyWeedtooth
                            24. Vinnie_Vegas
                                26. CashWho
                            25. MC_Fillius_Dickinson
            13. dmanny64
                19. TokeyWeedtooth
            

In [72]:
get_convo_details(threads[7597])

Subreddit: Marvel
twentyitalians
    thatgirlwithbigboobs
        pylestothemax
            trebory6
                pylestothemax
            ismailismail
    RockstarSuicide
        1stOnRt1
            oneELECTRIC
            screw_arc
                1stOnRt1
                    JasonSteakums
            RockstarSuicide
                1stOnRt1
                    RockstarSuicide
    Fanatical_Idiot
        TokeyWeedtooth
            Fanatical_Idiot
            da_bbq
                Martel732
                    [deleted]
                TokeyWeedtooth
            MC_Fillius_Dickinson
                TokeyWeedtooth
                    MC_Fillius_Dickinson
                        TokeyWeedtooth
                            Vinnie_Vegas
                                CashWho
                            MC_Fillius_Dickinson
            dmanny64
                TokeyWeedtooth
            RockstarSuicide
            FlatTire2005
    Mechfire11
        BeetIejewz
    Leo_TheLurker


In [70]:
random.sample(list(neg_threads7), 10)

[5399, 246, 4918, 9802, 7855, 4726, 6498, 5465, 5250, 4175]

In [71]:
get_convo_details(threads[4726])

Subreddit: hockey
Imagine1
    aschwan41
        Imagine1
            aschwan41
                Imagine1
            xdiagnosis
                Imagine1
            Yashkovich
                Imagine1
            camster1123
                Imagine1
            gruesome2some
            gruesome2some
            gruesome2some
            gruesome2some
            gruesome2some
            gruesome2some
    justinql


In [73]:
get_convo_details(threads[6498])

Subreddit: hockey
villagetorcher
    Podo13
        masterofstuff328
        chiddie
            Podo13
                chiddie
                DavidPuddy666
                themikeshow
            Roflcopter227
                Herpes_hurricane
                colt4594
                suck-me-beautiful
        Sirius_Cyborg
            nibblerhank
                sweetwattah
        Tsquare43
        MoreDescription
        Bigforsumthin
        sweetwattah
            BananApocalypse
    tandw
        [deleted]
    moak0
        philocity
        Oradi
    PaulClarkLoadletter
    thenicolino
